In [ ]:
import torch
from torch import nn


class GLUFFN(nn.Modules):
    def __init__(self, h1, h2, h3, hiddensize):
        super(GLUFFN, self).__init__()
        self.w1 = nn.Linear(hiddensize, h1)
        self.w2 = nn.Linear(hiddensize, h2)
        self.w3 = nn.Linear(hiddensize, h3)

    def forward(self, x):
        x1 = self.w1(x)
        x2 = self.w2(x)
        x2 = torch.matmul(x2, torch.sigmoid(x2))
        x3 = self.w3(torch.matmul(x1, x2))

        return x3

以下是 SwiGLU FFN（基于Swish门控的前馈网络）‌ 的完整 PyTorch 实现代码，结合了 Transformer 中常用的架构设计（类似 LLaMA 和 PaLM 的实现方式）：

# 完整代码实现‌
import torch
import torch.nn as nn
import torch.nn.functional as F

class SwiGLUFFN(nn.Module):
    def __init__(self, d_model: int, d_ff: int, dropout: float = 0.1):
         super().__init__()
        # SwiGLU 核心组件
        self.w1 = nn.Linear(d_model, d_ff)    # 门控分支的线性变换
        self.w2 = nn.Linear(d_model, d_ff)    # 值分支的线性变换
        self.w3 = nn.Linear(d_ff, d_model)     # 输出投影层
        self.dropout = nn.Dropout(dropout)    # 可选Dropout

        # 初始化参数（类似LLaMA的初始化方式）
        nn.init.normal_(self.w1.weight, std=0.02)
        nn.init.normal_(self.w2.weight, std=0.02)
        nn.init.normal_(self.w3.weight, std=0.02)
        nn.init.zeros_(self.w1.bias)
        nn.init.zeros_(self.w2.bias)
        nn.init.zeros_(self.w3.bias)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # SwiGLU门控计算
        gate = F.silu(self.w1(x))    # Swish(β=1) = SiLU
        value = self.w2(x)
        x = gate * value             # 门控融合

        # 输出投影 + Dropout
        return self.dropout(self.w3(x))

代码解析‌

架构设计‌：

输入/输出维度‌：d_model 是 Transformer 的隐藏层维度（例如 512），d_ff 是 FFN 的中间扩展维度（例如 2048）。
三线性层设计‌：
w1 和 w2 分别生成门控和值的中间表示。
w3 将 d_ff 维的特征投影回 d_model 维，保持输入输出维度一致。
参数初始化‌：使用小标准差正态分布初始化权重，偏置初始化为零（与大模型训练稳定性相关）。

其中 Swish 使用 PyTorch 内置的 F.silu 实现。门控融合后通过 W3 投影回原始维度。Dropout‌：可选丢弃层防止过拟合。

使用示例‌
python
Copy Code
# 定义输入 (batch_size=2, seq_len=10, d_model=512)
x = torch.randn(2, 10, 512)

# 初始化SwiGLU FFN (扩展维度d_ff=2048)
ffn = SwiGLUFFN(d_model=512, d_ff=2048)

# 前向传播
output = ffn(x)

# 验证输入输出维度一致
print(output.shape)  # torch.Size([2, 10, 512])

扩展：集成到Transformer Block‌
python
Copy Code
class TransformerBlock(nn.Module):
    def __init__(self, d_model: int, n_heads: int, d_ff: int):
        super().__init__()
        self.attention = nn.MultiheadAttention(d_model, n_heads)
        self.ffn = SwiGLUFFN(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # 自注意力残差连接
        attn_out, _ = self.attention(x, x, x)
        x = x + attn_out
        x = self.norm1(x)

        # FFN残差连接
        ffn_out = self.ffn(x)
        x = x + ffn_out
        x = self.norm2(x)
        return x

关键优势‌
更强的非线性‌：Swish门控比传统ReLU能捕获更复杂的特征交互。
训练稳定性‌：门控机制动态调整特征权重，缓解梯度消失。
兼容性‌：可直接替换标准Transformer中的FFN层（参数规模约为标准FFN的 2/3，需调整 d_ff 保持计算量一致）。

如需调整模型规模，只需修改 d_model 和 d_ff 的比例（例如 LLaMA-7B 使用 d_ff = 2.7 * d_model）。